# AAI Workshop 4

## EXAMPLE: Sampling from a probability distribution

All the algorithms for approximate inference with Bayesian Networks require a method for sampling from a known probability distribution.

Let's see a possible implementation of such sampling method in case of Boolean variables and known CPTs:

In [1]:
import numpy as np
import random as rnd

t, f = 0, 1

def samplegen(Pdist, Parents = []):
	assert len(Parents) < len(Pdist.shape)
	if rnd.random() < Pdist[t][tuple(Parents)]:
		return t
	return f

The input of this funtion is a probability distribution **Pdist** with the content of the CPT and, if available, the values of the **Parents** events.

Let's generate some samples from the Sprinkler network:

![Sprinkler example](sprinkler_dag_new.jpg)

In particular, let's say we want to generate 20 samples from the $Cloudy$ distribution, ${\bf P}(C)$, which has no parents:

In [2]:
P_C = np.array([0.5, 0.5])

for i in range(20):
    s = samplegen(P_C)
    print("C = ", 'TRUE' if s == t else 'FALSE')

C =  TRUE
C =  TRUE
C =  TRUE
C =  TRUE
C =  FALSE
C =  FALSE
C =  TRUE
C =  TRUE
C =  TRUE
C =  FALSE
C =  FALSE
C =  TRUE
C =  TRUE
C =  FALSE
C =  TRUE
C =  TRUE
C =  TRUE
C =  FALSE
C =  FALSE
C =  TRUE


As expected, there are approximately 50% $true$ and 50% $false$ outcomes.

Let's sample from a conditional distribution, for example ${\bf P}(R | \neg c)$:

In [3]:
P_R_C = np.array([[0.8, 0.1],[0.2, 0.9]])

for i in range(20):
    s = samplegen(P_R_C, [f])
    print("R = ", 'TRUE' if s == t else 'FALSE')

R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  FALSE
R =  TRUE
R =  FALSE
R =  FALSE


In this case there are many more $false$ than $true$ outcomes because ${\bf P}(R | \neg c) = \langle 0.1, 0.9\rangle$.

Finally, let's sample ${\bf P}(G | \neg s, r) = \langle 0.9, 0.1\rangle$:

In [4]:
P_G_SR = np.array([[[0.95, 0.9],[0.9, 0.1]],[[0.05, 0.1],[0.1, 0.9]]])

for i in range(20):
    s = samplegen(P_G_SR, [f, t])
    print("G = ", 'TRUE' if s == t else 'FALSE')

G =  TRUE
G =  FALSE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE
G =  TRUE


Even in this case, the samples are more or less as expected, about 90% $true$ and 10% $false$.

## EXAMPLE: Prior-Sampling

Let's implement now the *Prior Sampling* algorithm (see the lecture slides of *Bayesian Networks II* or Chapter 14 of Russell & Norvig's book), starting with some data structures to represent the Sprinkler network:

In [5]:
# some of these distributions were already defined before, but we repeate them just in case
P_C = np.array([0.5, 0.5])
P_S_C = np.array([[0.1, 0.5],[0.9, 0.5]])
P_R_C = np.array([[0.8, 0.1],[0.2, 0.9]])
P_G_SR = np.array([[[0.95, 0.9],[0.9, 0.1]],[[0.05, 0.1],[0.1, 0.9]]])

# network variables...
var = ['C','S','R','G']
# their distributions...
prd = {'C':P_C, 'S':P_S_C, 'R':P_R_C, 'G':P_G_SR}
# their parents...
par = {'C':[], 'S':['C'], 'R':['C'], 'G':['S','R']}
# and their initial values
val = {'C':f, 'S':f, 'R':f, 'G':f}


Let's define also a function to retrieve the values of the parents of a variable:

In [6]:
def parents(X):
	return [val[i] for i in par[X]]

The following algorithm generates 1000 events from the Sprinkler network:

In [7]:
event = []

for n in range(1000):
	for x in var:
		val[x] = samplegen(prd[x], parents(x))
	event.append(['f' if val[x] else 't' for x in var])

print("First randomly generated event = ", event[0])
print("Number or randomly generated events = ", len(event))

First randomly generated event =  ['t', 'f', 't', 't']
Number or randomly generated events =  1000


Finally, we can compute the probability of any event by counting the number of times it was generated and normalising. For example, we can verify that $P(c, \neg s, r, g) = 0.324 \approx \dfrac{N_{PS}(c, \neg s, r, g)}{N} = {\hat P}(c, \neg s, r, g)$, as predicted in the lecture:

In [8]:
P_query = event.count(['t', 'f', 't', 't']) / len(event)
print("P(c,¬s,r,g) = ", P_query)

P(c,¬s,r,g) =  0.318


which is indeed very close to the exact probability!

---

## EXERCISE: Burglary network

Implement the *Prior Sampling* algorithm to do approximate inference on last week's Burglary network.

![Burglary example](burglary_dag.jpg)

Verify that the algorithm can correctly approximate the probability $P(j, m, a, \neg b, \neg e) = 0.00063$

Try different numbers of samples (e.g. $N = 10, 100, 1000, 10000$) and compare the results.

---

## EXERCISE (optional)

Try to implement one of the following algorithms (see the pseudocode in the lecture's slides or the book):
- *Rejection Sampling*
- *Likelihood Weighting*
- *Gibbs Sampling*

Use your program to do approximate inference on the Sprinkler network and verify that ${\hat P}(c, \neg s, r, g) \approx 0.324$ and ${\hat P}(g | r, c) \approx 0.905$.

---

Write a short document (PDF, max 2 pages) or Jupyter Notebook file (preferred) describing your solution(s) and send it to **nbellotto@lincoln.ac.uk** with subject *AAI Workshop 4 - NAME SURNAME*. Please submit your work by the <u>15th Nov 2021</u>. **It will not be graded, but only used by the lecturer to check the progress of the class**.